# Deep Learning Challenge
## Michelle Rohrer

### Pakete laden

In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-09-05 14:40:33.085237: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Daten laden und vorbereiten
Die Daten wurden von kaggle heruntergeladen (https://www.kaggle.com/datasets/msambare/fer2013)

Infos zu den Daten:

Die Daten bestehen aus 48x48 Pixel großen Graustufenbildern von Gesichtern. Die Gesichter wurden automatisch so ausgerichtet, dass sie mehr oder weniger zentriert sind und ungefähr den gleichen Bereich in jedem Bild einnehmen.

Die Aufgabe besteht darin, jedes Gesicht anhand des im Gesichtsausdruck gezeigten Gefühls einer von sieben Kategorien zuzuordnen (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral). Der Trainingssatz besteht aus 28.709 Beispielen und der öffentliche Testsatz aus 3.589 Beispielen.

In [2]:
# schauen welche Dateien im Ordner sind
base_path = "/Users/husazwerg/.cache/kagglehub/datasets/msambare/fer2013/versions/1"
print(os.listdir(base_path))


['test', 'train']


In [3]:
base_path = "/Users/husazwerg/.cache/kagglehub/datasets/msambare/fer2013/versions/1"
train_dir = os.path.join(base_path, "train")
test_dir  = os.path.join(base_path, "test")

In [ ]:
img_size = 48
batch_size = 64

# Daten normalisieren -> Pixelwerte zwischen 0 und 1
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(train_dir,
                                             target_size=(img_size, img_size),
                                             batch_size=batch_size,
                                             class_mode='categorical')
test_generator = datagen.flow_from_directory(test_dir,
                                            target_size=(img_size, img_size),
                                            batch_size=batch_size,
                                            class_mode='categorical')

print(f"Klassen: {train_generator.class_indices}")


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Klassen: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
